In [5]:
#determinant 4 below used in final table in paper

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)]  
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_4']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_4', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]   # 768-dim BioClinicalBERT embeddings


# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 4)
treatment_columns = ['determinant_4']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_4.csv', index=False)


Class counts before balancing:
determinant_4:
determinant_4
False    328978
True       2815
Name: count, dtype: int64

Class counts after balancing for determinant_4:
determinant_4
0    328978
1    328978
Name: count, dtype: int64
20561/20561 [==============================] - 43s 2ms/step
Class counts in the final subsampled dataframe:
determinant_4:
determinant_4
1    5034
0    4966
Name: count, dtype: int64
Subgroup shape: (657925, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_4'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_4'],
      dtype='object', length=770)
Calculating for determinant_4
Class distr

In [6]:
#determinant 7 used in final table in paper

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)]  
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_7']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_7', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]   # 768-dim BioClinicalBERT embeddings


# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 7)
treatment_columns = ['determinant_7']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_7.csv', index=False)


Class counts before balancing:
determinant_7:
determinant_7
False    330170
True       1623
Name: count, dtype: int64

Class counts after balancing for determinant_7:
determinant_7
0    330170
1    330170
Name: count, dtype: int64
20635/20635 [==============================] - 43s 2ms/step
Class counts in the final subsampled dataframe:
determinant_7:
determinant_7
0    5034
1    4966
Name: count, dtype: int64
Subgroup shape: (660309, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_7'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_7'],
      dtype='object', length=770)
Calculating for determinant_7
Class distr

In [7]:
#determinant 6 did not  use this below result. we used determinant 6 value from previous siamese table

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)]  
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_6']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_6', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]   # 768-dim BioClinicalBERT embeddings

# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 6)
treatment_columns = ['determinant_6']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_6.csv', index=False)


Class counts before balancing:
determinant_6:
determinant_6
False    331760
True         33
Name: count, dtype: int64

Class counts after balancing for determinant_6:
determinant_6
0    331760
1    331760
Name: count, dtype: int64
20735/20735 [==============================] - 47s 2ms/step
Class counts in the final subsampled dataframe:
determinant_6:
determinant_6
1    5015
0    4985
Name: count, dtype: int64
Subgroup shape: (663489, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_6'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_6'],
      dtype='object', length=770)
Calculating for determinant_6
Class distr

In [8]:
#determinant 5 we did not use this below result.we used old siamese results for final table

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)]  
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_5']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_5', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]   

# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 5)
treatment_columns = ['determinant_5']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_5.csv', index=False)


Class counts before balancing:
determinant_5:
determinant_5
False    331682
True        111
Name: count, dtype: int64

Class counts after balancing for determinant_5:
determinant_5
0    331682
1    331682
Name: count, dtype: int64
20730/20730 [==============================] - 47s 2ms/step
Class counts in the final subsampled dataframe:
determinant_5:
determinant_5
0    5016
1    4984
Name: count, dtype: int64
Subgroup shape: (663333, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_5'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_5'],
      dtype='object', length=770)
Calculating for determinant_5
Class distr

In [9]:
#determinant 10 we used this below result for our final table

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)]  
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_10']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_10', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist() 
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]   


# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 10)
treatment_columns = ['determinant_10']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_10.csv', index=False)


Class counts before balancing:
determinant_10:
determinant_10
True     328834
False      2959
Name: count, dtype: int64

Class counts after balancing for determinant_10:
determinant_10
1    328834
0    328834
Name: count, dtype: int64
20552/20552 [==============================] - 48s 2ms/step
Class counts in the final subsampled dataframe:
determinant_10:
determinant_10
1    5045
0    4955
Name: count, dtype: int64
Subgroup shape: (657637, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_10'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_10'],
      dtype='object', length=770)
Calculating for determinant_10
Cl

In [4]:
#determinant 12 we used below result for determinant 12 in our paper final

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)]  
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_12']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_12', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]
# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 12)
treatment_columns = ['determinant_12']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_12.csv', index=False)


Class counts before balancing:
determinant_12:
determinant_12
False    252156
True      79637
Name: count, dtype: int64

Class counts after balancing for determinant_12:
determinant_12
1    252156
0    252156
Name: count, dtype: int64
15759/15759 [==============================] - 34s 2ms/step
Class counts in the final subsampled dataframe:
determinant_12:
determinant_12
1    5095
0    4905
Name: count, dtype: int64
Subgroup shape: (504263, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_12'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_12'],
      dtype='object', length=770)
Calculating for determinant_12
Cl

In [12]:
#determinant 8 we used this below results for final paper

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)] 
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_8']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_8', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]

# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 8)
treatment_columns = ['determinant_8']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_8.csv', index=False)


Class counts before balancing:
determinant_8:
determinant_8
False    183286
True     148507
Name: count, dtype: int64

Class counts after balancing for determinant_8:
determinant_8
0    183286
1    183286
Name: count, dtype: int64
11455/11455 [==============================] - 27s 2ms/step
Class counts in the final subsampled dataframe:
determinant_8:
determinant_8
0    5033
1    4967
Name: count, dtype: int64
Subgroup shape: (366541, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_8'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_8'],
      dtype='object', length=770)
Calculating for determinant_8
Class distr

In [14]:
#determinant 9 we used this below resulst for final paper

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)] 
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_9']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_9', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]
# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 9)
treatment_columns = ['determinant_9']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_9.csv', index=False)


Class counts before balancing:
determinant_9:
determinant_9
True     257381
False     74412
Name: count, dtype: int64

Class counts after balancing for determinant_9:
determinant_9
1    257381
0    257381
Name: count, dtype: int64
16086/16086 [==============================] - 37s 2ms/step
Class counts in the final subsampled dataframe:
determinant_9:
determinant_9
0    5036
1    4964
Name: count, dtype: int64
Subgroup shape: (514729, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_9'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_9'],
      dtype='object', length=770)
Calculating for determinant_9
Class distr

In [17]:
#determinant1 not used this below results.used results from old table only

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)]  
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_1']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_1', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]
# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 1)
treatment_columns = ['determinant_1']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_1.csv', index=False)


Class counts before balancing:
determinant_1:
determinant_1
False    331531
True        262
Name: count, dtype: int64

Class counts after balancing for determinant_1:
determinant_1
0    331531
1    331531
Name: count, dtype: int64
20720/20720 [==============================] - 45s 2ms/step
Class counts in the final subsampled dataframe:
determinant_1:
determinant_1
0    5015
1    4985
Name: count, dtype: int64
Subgroup shape: (663031, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_1'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_1'],
      dtype='object', length=770)
Calculating for determinant_1
Class distr

In [19]:
#determinant11 not used this below results.used new results from windows mach

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)] 
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_11']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_11', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]
# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 11)
treatment_columns = ['determinant_11']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_11.csv', index=False)


Class counts before balancing:
determinant_11:
determinant_11
True     285171
False     46622
Name: count, dtype: int64

Class counts after balancing for determinant_11:
determinant_11
1    285171
0    285171
Name: count, dtype: int64
17823/17823 [==============================] - 40s 2ms/step
Class counts in the final subsampled dataframe:
determinant_11:
determinant_11
1    5017
0    4983
Name: count, dtype: int64
Subgroup shape: (570308, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_11'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_11'],
      dtype='object', length=770)
Calculating for determinant_11
Cl

In [21]:
#determinant2 not used these below results.we used new results from windows mach

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)]  
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_2']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_2', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]
# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 2)
treatment_columns = ['determinant_2']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_2.csv', index=False)


Class counts before balancing:
determinant_2:
determinant_2
True     328694
False      3099
Name: count, dtype: int64

Class counts after balancing for determinant_2:
determinant_2
1    328694
0    328694
Name: count, dtype: int64
20543/20543 [==============================] - 49s 2ms/step
Class counts in the final subsampled dataframe:
determinant_2:
determinant_2
0    5020
1    4980
Name: count, dtype: int64
Subgroup shape: (657357, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_2'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_2'],
      dtype='object', length=770)
Calculating for determinant_2
Class distr

In [23]:
#determinant3 not used these below results.we used new results from windows mach

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)]  
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_3']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_3', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]
# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 3)
treatment_columns = ['determinant_3']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_3.csv', index=False)


Class counts before balancing:
determinant_3:
determinant_3
False    330722
True       1071
Name: count, dtype: int64

Class counts after balancing for determinant_3:
determinant_3
0    330722
1    330722
Name: count, dtype: int64
20670/20670 [==============================] - 48s 2ms/step
Class counts in the final subsampled dataframe:
determinant_3:
determinant_3
1    5100
0    4900
Name: count, dtype: int64
Subgroup shape: (661414, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_3'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_3'],
      dtype='object', length=770)
Calculating for determinant_3
Class distr

In [25]:
#determinant 13

In [ ]:
# Import necessary libraries
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
import statsmodels.api as sm
import warnings
import gc

# Step 1: Data Loading and SMOTE Balancing
# Load your data in chunks
chunk_size = 100000
chunks = pd.read_csv('predictions_with_embeddings_sampled.csv', chunksize=chunk_size)
df = pd.concat(chunks, ignore_index=True)
del chunks  # Free memory

# Ensure that the outcome column is numeric
df['opioid_pr_ab'] = pd.to_numeric(df['opioid_pr_ab'], errors='coerce').astype(int)

# Features including embedding columns
embedding_columns = [str(i) for i in range(768)]  
features = embedding_columns

# Initialize SMOTE for class balancing
smote = SMOTE(sampling_strategy='minority', random_state=42)
cols_to_balance = ['determinant_13']

# Print class counts before balancing
print("Class counts before balancing:")
for col in cols_to_balance:
    if col in df.columns:
        print(f"{col}:")
        print(df[col].value_counts())

# Apply SMOTE to balance classes
balanced_data_list = []
for col in cols_to_balance:
    if col in df.columns:
        df_features = df[features + ['opioid_pr_ab']]  # Include the 'opioid_pr_ab' column for SMOTE
        df_target = df[col].astype(int)
        df_features_balanced, df_target_balanced = smote.fit_resample(df_features, df_target)
        
        # Combine the balanced features and target back into a dataframe
        df_balanced = pd.concat([df_features_balanced, pd.Series(df_target_balanced, name=col)], axis=1)
        balanced_data_list.append(df_balanced)

        # Print class counts after balancing for each determinant
        print(f"\nClass counts after balancing for {col}:")
        print(pd.Series(df_target_balanced).value_counts())

        # Clear memory
        del df_features, df_target, df_features_balanced, df_target_balanced, df_balanced
        gc.collect()

# Combine the balanced dataframes
df_balanced_final = pd.concat(balanced_data_list, axis=0).drop_duplicates().reset_index(drop=True)
del balanced_data_list  # Free memory

# Ensure no duplicate columns after merging
df_balanced_final = df_balanced_final.loc[:, ~df_balanced_final.columns.duplicated()]

# Step 2: Create Siamese Neural Network Model
def create_siamese_nn(input_dim, hidden_dim, dropout_prob):
    x = Input(shape=(input_dim,), name='x')
    shared = Dense(hidden_dim, activation='relu')(x)
    shared = Dropout(dropout_prob)(shared)
    t1 = Input(shape=(1,), name='t1')
    t1_shared = Dense(hidden_dim, activation='relu')(t1)
    t1_shared = Dropout(dropout_prob)(t1_shared)
    t1_output = Dense(1, activation='linear')(Concatenate()([shared, t1_shared]))
    t0 = Input(shape=(1,), name='t0')
    t0_shared = Dense(hidden_dim, activation='relu')(t0)
    t0_shared = Dropout(dropout_prob)(t0_shared)
    t0_output = Dense(1, activation='linear')(Concatenate()([shared, t0_shared]))
    model = Model(inputs=[x, t0, t1], outputs=[t0_output, t1_output])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Subgroup identification using Siamese Neural Network
def subgroup_identification(data, treatment_col, outcome_col, features, hidden_dim, dropout_prob, epochs, k):
    t0_data = data[data[treatment_col] == 0].copy()
    t1_data = data[data[treatment_col] == 1].copy()
    mx_dim = min(t0_data.shape[0], t1_data.shape[0])
    t0_data = t0_data.head(mx_dim)
    t1_data = t1_data.head(mx_dim)
    siamese_nn = create_siamese_nn(len(features), hidden_dim, dropout_prob)
    siamese_nn.fit([t0_data[features], t0_data[treatment_col], t1_data[treatment_col]], [t0_data[outcome_col], t1_data[outcome_col]], epochs=epochs, verbose=0)
    t0_effects, t1_effects = siamese_nn.predict([data[features], data[treatment_col], data[treatment_col]])
    abs_effects = abs(t1_effects - t0_effects)
    subgroup = data[abs_effects > k]
    return subgroup

# Identify the subgroup using the Siamese Neural Network
hidden_dim = 200
dropout_prob = 0.5
epochs = 100
k = 0.72
subgroup = subgroup_identification(df_balanced_final, 'determinant_13', 'opioid_pr_ab', features, hidden_dim, dropout_prob, epochs, k)

# Remove duplicate rows from the subgroup
subgroup = subgroup.drop_duplicates().reset_index(drop=True)

# Subsample 10,000 data points from the identified subgroup
subsampled_df2 = subgroup.sample(n=10000, random_state=42) if len(subgroup) > 10000 else subgroup

# Handle NaN or infinite values before converting to integers
subsampled_df2[cols_to_balance] = subsampled_df2[cols_to_balance].fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# Check the class counts in the final subsampled dataframe
print("Class counts in the final subsampled dataframe:")
for col in cols_to_balance:
    if col in subsampled_df2.columns:
        print(f"{col}:")
        print(subsampled_df2[col].value_counts())

print(f"Subgroup shape: {subgroup.shape}")
print(f"Subsampled dataframe shape: {subsampled_df2.shape}")

# Print the columns in the subgroup
print("Columns in the subgroup:")
print(subgroup.columns)

# Print the columns in the subsampled dataframe
print("Columns in the subsampled dataframe:")
print(subsampled_df2.columns)

# Step 3: Causal Inference Calculation
# Ensure that the outcome column is numeric in subsampled_df2
subsampled_df2['opioid_pr_ab'] = pd.to_numeric(subsampled_df2['opioid_pr_ab'], errors='coerce').astype(int)

# Define the outcome and confounders
outcome = 'opioid_pr_ab'
#embedding_columns = subsampled_df2.columns[13:].tolist()  
#confounders = embedding_columns
confounders  = [str(i) for i in range(768)]   # 768-dim BioClinicalBERT embeddings


# Function to calculate propensity scores
def calculate_propensity_scores(df, treatment, confounders):
    X = df[confounders].values
    y = df[treatment]
    if len(y.unique()) == 2:
        model = LogisticRegression(max_iter=5000)
        model.fit(X, y)
        propensity_scores = model.predict_proba(X)[:, 1]
        return propensity_scores
    else:
        raise ValueError(f"The target variable '{treatment}' is not binary.")

# Function to run propensity score matching and calculate ATE
def run_ps(sampled_df, confounders, treatment, outcome):
    X_data = sampled_df[confounders].values
    y_data = sampled_df[outcome]
    ps = LogisticRegression(max_iter=5000, C=1e6, n_jobs=-1).fit(X_data, sampled_df[treatment]).predict_proba(X_data)[:, 1]
    weight = (sampled_df[treatment] - ps) / (ps * (1 - ps))
    return np.mean(weight * sampled_df[outcome])

# Function to calculate CATE
def calculate_cate(sampled_df, treatment, outcome):
    treated_df = sampled_df[sampled_df[treatment] == 1]
    untreated_df = sampled_df[sampled_df[treatment] == 0]
    cate = treated_df[outcome].mean() - untreated_df[outcome].mean()
    return cate

# Initialize a list to store the results
results = []

# Suppress specific warnings
warnings.filterwarnings("ignore", message="lbfgs failed to converge")
warnings.filterwarnings("ignore", message="Pandas requires version")

# Drop the determinant_pr_ab column if it exists
if 'determinant_pr_ab' in subsampled_df2.columns:
    subsampled_df2 = subsampled_df2.drop(columns=['determinant_pr_ab'])

# Define the treatment column (only determinant 13)
treatment_columns = ['determinant_13']

# Loop through each determinant
for treatment in treatment_columns:
    print(f"Calculating for {treatment}")
    subsampled_df2[treatment] = pd.to_numeric(subsampled_df2[treatment], errors='coerce')
    
    # Check class distribution
    class_counts = subsampled_df2[treatment].value_counts()
    print(f"Class distribution for {treatment}:")
    print(class_counts)
    
    try:
        if len(class_counts) == 2:
            subsampled_df2['propensity_score'] = calculate_propensity_scores(subsampled_df2, treatment, confounders)
            treated_df = subsampled_df2[subsampled_df2[treatment] == 1]
            untreated_df = subsampled_df2[subsampled_df2[treatment] == 0]
            weight_t = 1 / treated_df["propensity_score"]
            weight_nt = 1 / (1 - untreated_df["propensity_score"])
            y1 = sum(treated_df[outcome] * weight_t) / len(weight_t)
            y0 = sum(untreated_df[outcome] * weight_nt) / len(weight_nt)
            bootstrap_sample = 1000

            # Define a function to run within the parallel loop
            def run_parallel(sample_idx):
                sample = subsampled_df2.sample(frac=1, replace=True, random_state=sample_idx).reset_index(drop=True)
                return run_ps(sample, confounders, treatment, outcome)

            # Run bootstrap samples in parallel
            ates = Parallel(n_jobs=-1)(
                delayed(run_parallel)(sample_idx) for sample_idx in range(bootstrap_sample)
            )
            ates = np.array(ates)
            ci_lower = np.percentile(ates, 2.5)
            ci_upper = np.percentile(ates, 97.5)
            ATE = np.mean(ates)
            CATE = calculate_cate(subsampled_df2, treatment, outcome)
            model = sm.OLS(subsampled_df2[outcome], sm.add_constant(subsampled_df2[[treatment, 'propensity_score']].astype(float)))
            result = model.fit()
            p_value = result.pvalues[treatment]
            original_sample_size = len(subsampled_df2)
            treated_sample_size = len(treated_df)
            untreated_sample_size = len(untreated_df)
            results.append({
                'Determinant': treatment,
                'Original Sample Size': original_sample_size,
                'Treated Sample Size': treated_sample_size,
                'Untreated Sample Size': untreated_sample_size,
                'Y1': y1,
                'Y0': y0,
                'ATE': ATE,
                'CATE': CATE,
                'p-value': p_value,
                '95% CI Lower': ci_lower,
                '95% CI Upper': ci_upper
            })
        else:
            print(f"Skipping {treatment}: This solver needs samples of at least 2 classes in the data, but the data contains only one class: {class_counts.index[0]}")
    except ValueError as e:
        print(f"Skipping {treatment}: {e}")

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('causal_inference_results_determinant_13.csv', index=False)


Class counts before balancing:
determinant_13:
determinant_13
True     311212
False     20581
Name: count, dtype: int64

Class counts after balancing for determinant_13:
determinant_13
1    311212
0    311212
Name: count, dtype: int64
19449/19449 [==============================] - 44s 2ms/step
Class counts in the final subsampled dataframe:
determinant_13:
determinant_13
0    5016
1    4984
Name: count, dtype: int64
Subgroup shape: (622367, 770)
Subsampled dataframe shape: (10000, 770)
Columns in the subgroup:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_13'],
      dtype='object', length=770)
Columns in the subsampled dataframe:
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'opioid_pr_ab',
       'determinant_13'],
      dtype='object', length=770)
Calculating for determinant_13
Cl